In [ ]:
##### Priklad 1: word count ------------------

# nacteni RDD
lines = sc.textFile("/user/pascepet/data/bible/bible.txt")

# rozdeleni radku
words = lines.flatMap(lambda line: line.split(" "))

# transformace radku na (key, value)
pairs = words.map(lambda word: (word, 1))

# secteni jednicek ke kazdemu klici
counts = pairs.reduceByKey(lambda a, b: a + b)

# vypis vysledku
counts.take(5)


In [ ]:
##### Priklad 2: vytazeni metadat z EXIF obrazku ------------

# knihovny
from PIL import Image
from PIL.ExifTags import TAGS
import StringIO

# funkce na extrakci EXIF z binarnich dat
def get_exif(str):
    ret = {}
    fn = StringIO.StringIO(str)
    i = Image.open(fn)
    info = i._getexif()
    if info:
        for tag, value in info.items():
            decoded = TAGS.get(tag, tag)
            ret[decoded] = value
    return ret

# funkce na transformaci dat EXIF
def exif_data_transform(exif_data):
    ret = {}
    ret['datetime'] = exif_data['DateTime'] if exif_data.has_key('DateTime') else ''
    ret['doba_expozice'] = exif_data['ExposureTime'][0] * 1.0 / exif_data['ExposureTime'][1] \
        if exif_data.has_key('ExposureTime') else 0.0
    ret['sirka'] = exif_data['ExifImageWidth'] if exif_data.has_key('ExifImageWidth') else 0
    ret['vyska'] = exif_data['ExifImageHeight'] if exif_data.has_key('ExifImageHeight') else 0
    ret['orientace'] = exif_data['Orientation'] if exif_data.has_key('Orientation') else 1
    return ret

# nacteme z HDFS adresar s obrazky, vraci list dvojic (jmeno souboru, binarni kod)
imgs = sc.binaryFiles('/user/pascepet/data/images/')

# transformace: z obrazku se ponecha jen jmeno souboru a data EXIF jako dict tag-value
imgs2 = imgs.map(lambda im: (im[0], get_exif(im[1])))

# ... dalsi akce ...
# napr. ponechani jen urcitych tagu a jejich uprava
imgs3 = imgs2.map(lambda im: (im[0], exif_data_transform(im[1])))

# ulozeni RDD na HDFS
imgs3.saveAsTextFile('/user/pascepet/data/images_stat')